In [1]:
!pip install pyspark==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=df93d2f9c753842d6b6fca9abc2bc9839e47573f7592cf0d6a9c2d0e4b79fcf3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
# Conectando ao google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.appName("MinMaxScaler").getOrCreate()

In [7]:
from pyspark.ml.feature import MinMaxScaler

In [8]:
carros = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Spark/datas/Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="vetor")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo", "Cilindros", "Cilindradas", "vetor").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|             vetor|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [10]:
escala = MinMaxScaler(inputCol="vetor", outputCol="minmaxscaler", min=0, max=1)
modelo = escala.fit(carros_vetor)
carrosminmax = modelo.transform(carros_vetor)
carrosminmax.select("vetor", "minmaxscaler").show(truncate=False)

+------------------+-----------------------------------------------+
|vetor             |minmaxscaler                                   |
+------------------+-----------------------------------------------+
|[21.0,6.0,160.0]  |[0.018518518518518517,0.5,0.030235162374020158]|
|[21.0,6.0,160.0]  |[0.018518518518518517,0.5,0.030235162374020158]|
|[228.0,4.0,108.0] |[0.6574074074074073,0.0,0.010824934677118328]  |
|[214.0,6.0,258.0] |[0.6141975308641975,0.5,0.06681597611048899]   |
|[187.0,8.0,360.0] |[0.5308641975308641,1.0,0.10488988428518103]   |
|[181.0,6.0,225.0] |[0.5123456790123456,0.5,0.05449794699514744]   |
|[143.0,8.0,360.0] |[0.3950617283950617,1.0,0.10488988428518103]   |
|[244.0,4.0,1467.0]|[0.70679012345679,0.0,0.5181037700634565]      |
|[228.0,4.0,1408.0]|[0.6574074074074073,0.0,0.49608062709966405]   |
|[192.0,6.0,1676.0]|[0.5462962962962963,0.5,0.5961179544606197]    |
|[178.0,6.0,1676.0]|[0.5030864197530864,0.5,0.5961179544606197]    |
|[164.0,8.0,2758.0]|[0.45987654320